# This Script transforms a .ttl file with URIS containing ids to a .txt file in the "G-Care" format


In [1]:
import os
import json
import random
from tqdm import tqdm

In [3]:
# Sets of entities and predicates
vertices = set()
predicates = set()

In [4]:
# Declare which URI corresponds to rdf:type
#rdf_type_uri = "<http://example.com/13000179>" # Yago
rdf_type_uri = "<http://example.org/1>" # Lubm
#rdf_type_uri = '<http://ex.org/03>' # SWDF

In [5]:
# Add all entities and predicates from the .ttl file
l = 0
ttl_file = open("/home/tim/cardinality_estimator/Datasets/lubm/graph/lubm.ttl", "r")
for line in tqdm(ttl_file):
    atoms = line.split(" ")[:-1]
    #if not atoms == []:
    if True:
        l += 1
        vertices.add(atoms[0])
        vertices.add(atoms[2])
        predicates.add(atoms[1])

2688849it [00:02, 1327015.41it/s]


In [6]:
# We need to map the URL ids to entity and predicate ids
id_to_id_mapping = {}
id_to_id_mapping_predicate = {}

In [7]:
# Creating Vertex Dict and save entity mappings
vertex_dict = {}
vid = 0
for vertex in tqdm(vertices):
    dvid = vertex.split("/")[-1].replace(">", "")
    #vertex_dict[vertex] = [dvid]
    vertex_dict[vertex] = [vid]
    id_to_id_mapping[vertex] = vid
    vid += 1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 664048/664048 [00:00<00:00, 762881.25it/s]


In [8]:
# Saving Mappings for Predicates
pid = 0
for p in predicates:
    id_to_id_mapping_predicate[p] = pid
    pid += 1

In [9]:
# Add Vertex types
ttl_file = open("/home/tim/cardinality_estimator/Datasets/lubm/graph/lubm.ttl", "r")
for line in tqdm(ttl_file):
    atoms = line.split(" ")[:-1]
    # If triple has predicate rdf:type
    if atoms[1] == rdf_type_uri:
        vertex_dict[atoms[0]] += vertex_dict[atoms[2]]


2688849it [00:01, 1886551.81it/s]


In [10]:
# Add Default Label if entity has no types:
for v in vertex_dict:
    if len(vertex_dict[v]) == 1:
        vertex_dict[v].append(0)

In [11]:
# Creating Edge List
edge_list = []
ttl_file = open("/home/tim/cardinality_estimator/Datasets/lubm/graph/lubm.ttl", "r")
for tp in tqdm(ttl_file):
    tp = tp.split(" ")[:-1]
    #edge_label = tp[1].split("/")[-1].replace(">", "") if not "?" in tp[1] else -1
    edge_list.append([vertex_dict[tp[0]][0], vertex_dict[tp[2]][0], id_to_id_mapping_predicate[tp[1]]])

2688849it [00:04, 662380.89it/s]


In [13]:
# Writing the Data File
with open("/home/tim/gcare/data/dataset/yago/yago.txt", "w") as f:
    f.write("t # 1")
    f.write("\n")
    for v in tqdm(vertex_dict):
        f.write("v")
        for p in vertex_dict[v]:
            f.write(" ")
            f.write(str(p) + "")
        #f.write("v " + str(vertex_dict[v][0]) + " ")
        f.write("\n")
    for e in tqdm(edge_list):
        f.write("e " + str(e[0]) + " " + str(e[1]) + " " + str(e[2]))
        f.write("\n")

100%|██████████████████████████| 58276870/58276870 [00:44<00:00, 1314373.99it/s]


In [14]:
# Save to ID to ID mapping for later query transformation
with open("/home/tim/cardinality_estimator/Datasets/yago/id_to_id_mapping.json", "w") as f:
    json.dump(id_to_id_mapping, f)
    
with open("/home/tim/cardinality_estimator/Datasets/yago/id_to_id_mapping_predicate.json", "w") as f:
    json.dump(id_to_id_mapping_predicate, f)

# End 

In [72]:
for e in edge_list:
    #try:
    assert ("http://example.com/" + e[0]) in vertex_dict
    ##except:
      #  print("http://example.com/" + e[0])
    assert ("http://example.com/" + e[1]) in vertex_dict

